In [21]:
import pandas as pd



In [22]:
df = pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [23]:
df=df.drop(["RowNumber","CustomerId","Surname"],axis =1)

In [24]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

label_encoder_gender = LabelEncoder()
df["Gender"] = label_encoder_gender.fit_transform(df["Gender"])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [25]:
##one hot encoding "geography"

onehot_encoder_geo = OneHotEncoder(handle_unknown="ignore")
geo_encoded = onehot_encoder_geo.fit_transform(df[["Geography"]]).toarray()
geo_encoded_dataframe =pd.DataFrame(
    geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(["Geography"]))
geo_encoded_dataframe

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [26]:
df = pd.concat([df.drop("Geography",axis=1),geo_encoded_dataframe],axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [27]:
x = df.drop("EstimatedSalary",axis=1)
y = df["EstimatedSalary"]

In [28]:
##train and testing 
from sklearn.model_selection import train_test_split
x_train,x_test,y_train_,y_test = train_test_split(
    x,y,test_size=0.2,random_state=42)

In [29]:
##stansdard scaler 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [30]:
##save models 
import pickle
with open("label_encoder_gender.pkl","wb") as file:
    pickle.dump(label_encoder_gender,file)

with open("onehot_encode_geo.pkl","wb") as file:
    pickle.dump(onehot_encoder_geo,file)

with open("scaler.pkl","wb") as file:
    pickle.dump(scaler,file)

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

##build the model 
model = Sequential(
    [
        Dense(64,activation="relu",input_dim = (x_train_scaled.shape[1])),
        Dense(32,activation="relu"),
        Dense(1)
    ]
)

##modle complie 
model.compile(optimizer = "adam",loss ="mean_absolute_error",
              metrics =["mae"])
model.summary()

c:\Users\jaikr\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [32]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

##set up tensorboard 
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback= TensorBoard(log_dir = log_dir,histogram_freq = 1)

##set up Earlystopping 

early_stopping = EarlyStopping(
    monitor = "val_loss",
    patience = 10,
    restore_best_weights = True
)

history = model.fit(
    x_train_scaled,y_train_,
    validation_data = (x_test_scaled,y_test),
    epochs = 100,
    callbacks = [early_stopping,tensorboard_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 100975.3906 - mae: 100975.3906 - val_loss: 98524.4531 - val_mae: 98524.4531
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 99719.4688 - mae: 99719.4688 - val_loss: 97129.4609 - val_mae: 97129.4609
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 98616.1016 - mae: 98616.1016 - val_loss: 93490.1875 - val_mae: 93490.1875
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 94676.1797 - mae: 94676.1797 - val_loss: 87243.1094 - val_mae: 87243.1094
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 87771.5938 - mae: 87771.5938 - val_loss: 79043.9922 - val_mae: 79043.9922
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 78094.9141 - mae: 78094.9141 - val_loss: 70304.2266 - val_mae: 70304.2266
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 69123.8750 - mae: 69123.8750 - val_loss: 62399.6602 - val_mae: 62399.6602
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 

In [33]:
##evaluate model on test data 
test_loss , test_mae = model.evaluate(x_test,y_test)
print(f"Test MAE : {test_mae}" )

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1382031744.0000 - mae: 1382031744.0000  
Test MAE : 1350461696.0


In [34]:
model.save("regression_model.h5")